In [1]:
! pip install neptune-client

     |████████████████████████████████| 315 kB 2.3 MB/s eta 0:00:01     |███████████████▋                | 153 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 4.3 MB/s  eta 0:00:01
  Created wheel for neptune-client: filename=neptune_client-0.15.2-py2.py3-none-any.whl size=565172 sha256=fee09564d06accc8af6f701b59262ce59c074d818fda9677f09eaa89a8f2a49b
  Stored in directory: /root/.cache/pip/wheels/4f/ff/e6/2128c7fd2e85bd327f524d44b07922848c40e48de6e75baa7e
  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7-py3-none-any.whl size=18149 sha256=0e41fa8a76cf418c1c45f234e012d3e59ff24a526498bfdf58463a2b3de96314
  Stored in directory: /root/.cache/pip/wheels/f3/1d/9f/2a74caecb81b8beb9a4fbe1754203d4b7cf42ef5d39e0d2311
Successfully built neptune-client strict-rfc3339


In [2]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6034  100  6034    0     0  79880      0 --:--:-- --:--:-- --:--:-- 80453
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
Found existing installation: torch 1.7.1+cpu
Uninstalling torch-1.7.1+cpu:
  Successfully uninstalled torch-1.7.1+cpu
Found existing installation: torchvision 0.8.2+cpu
Uninstalling torchvision-0.8.2+cpu:
  Successfully uninstalled torchvision-0.8.2+cpu
Copying gs://tpu-pytorch/wheels/torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl...
\ [1 files][ 91.0 MiB/ 91.0 MiB]                                                
Operation completed over 1 objects/91.0 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200515-cp37-cp37m-linux_x86_6

In [3]:
import glob
import tensorflow as tf
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import albumentations
from PIL import Image
import io
import matplotlib.pyplot as plt
import neptune.new as neptune
import torch.optim as optim
import torchvision.models as models
import numpy as np
import torch_xla
import torch_xla.core.xla_model as xm
from torchvision import transforms

2022-03-22 18:55:46.135994: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-03-22 18:55:46.136104: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[('__call__', <function LevelMapper.__call__ at 0x7ff1b30ff710>), ('__init__', <function LevelMapper.__init__ at 0x7ff1b30ff680>)]
[('__call__', <function BalancedPositiveNegativeSampler.__call__ at 0x7ff1b2fcc050>), ('__init__', <function BalancedPositiveNegativeSampler.__init__ at 0x7ff1b3046f80>)]
[('__init__', <function BoxCoder.__init__ at 0x7ff1b2fd4830>), ('decode', <function BoxCoder.decode at 0x7ff1b2fd49e0>), ('decode_single', <function BoxCoder.decode_single at 0x7ff1b2fd4a70>), ('encode', <function BoxCoder.encode at 0x7ff1b2fd48c0>), ('encode_single', <function BoxCoder.encode_single at 0x7ff1b2fd4950>)]
[('__call__', <function Matcher.__call__ at 0x7ff1b2fd45f0>), ('__init__', <function Matcher.__init__ at 0x7ff1b2fd4b90>), ('set_low_quality_matches_', <function Matcher.set_low_quality_matches_ at 0x7ff1b2fd47a0>)]
[('__init__', <function ImageList.__init__ at 0x7ff1b2fd4c20>), ('to', <function ImageList.to at 0x7ff1b2fd4cb0>)]
[('__init__', <function Timebase.__init__ at

In [18]:
train_files = glob.glob('../input/tpu-getting-started/tfrecords-jpeg-512x512/train/*.tfrec')
val_files = glob.glob('../input/tpu-getting-started/tfrecords-jpeg-512x512/val/*.tfrec')

In [19]:
feature_description = {
    'id': tf.io.FixedLenFeature([], tf.string),
    'class': tf.io.FixedLenFeature([], tf.int64),
    'image': tf.io.FixedLenFeature([], tf.string)
}



In [20]:
class PetalDataset(Dataset):
    def __init__(self,image_files,aug):
        self.image_files = image_files
        self.features = {
            'id': tf.io.FixedLenFeature([], tf.string),
            'class': tf.io.FixedLenFeature([], tf.int64),
            'image': tf.io.FixedLenFeature([], tf.string)
            }
        self.image_dataset = tf.data.TFRecordDataset(self.image_files)
        self.image_dataset = self.image_dataset.map(self._parse_function)
        self.indexed_data = self._create_indexed_data(self.image_dataset)
        self.aug = aug
        self.to_tensor = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    def _parse_function(self,example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
      return tf.io.parse_single_example(example_proto, feature_description)
    
    def _create_indexed_data(self,image_dataset):
        indexed_data = []
        for image_data in image_dataset:
            clazz = int(image_data['class'].numpy())
            id = str(image_data['id'].numpy())[2:-1]
            image = image_data['image'].numpy()
            indexed_data.append({'class':clazz,'id':id,'img':image})
        return indexed_data
    
    def __len__(self):
        return len(self.indexed_data)
        
    def __getitem__(self,index):
        item = self.indexed_data[index]
        img = item['img']
        img = np.array(Image.open(io.BytesIO(img)))
        img = self.aug(image = img)['image']
        img = self.to_tensor(img)
        return img,item['class']    
        

In [21]:
train_aug = albumentations.Compose([
            albumentations.RandomResizedCrop(224,224),
            albumentations.HorizontalFlip(),
            albumentations.VerticalFlip(),
            albumentations.Rotate(10),
            albumentations.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.25),
            albumentations.CoarseDropout(max_holes=6, max_height=10, max_width=10),
        ])
val_aug = albumentations.Compose([
    albumentations.CenterCrop(224,224)
])

In [22]:
train_dataset = PetalDataset(train_files,aug=train_aug)
val_dataset=PetalDataset(val_files,aug=val_aug)

In [23]:
train_dl = DataLoader(train_dataset,batch_size=200,shuffle=True)
val_dl = DataLoader(val_dataset,batch_size=200,shuffle=False)

In [24]:
class PetalModel(nn.Module):
    def __init__(self,pre_trained=True):
        super(PetalModel,self).__init__()
        self.pre_trained=pre_trained
        self.model= models.densenet121(pretrained=self.pre_trained)
        #self.model = models.resnet34(pretrained=self.pre_trained)
        self.model.classifier = nn.Linear(self.model.classifier.in_features,104)
    def forward(self,x):
        return self.model(x)

In [25]:
#device = torch.device('cuda')
device = xm.xla_device()

In [26]:
model = PetalModel()

In [27]:
params = {'lr':3e-4, 
           'momentum':0.9, 
           'iterations':5}
optimizer = torch.optim.Adam(model.parameters())

In [28]:
run = neptune.init_project(name="njoysubho/petal", api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJjZDE1N2MzMC1mYzBkLTRjM2UtOTc0OC1iZWFlM2MwMGY4NjgifQ==")
run['parameters'] = params

https://app.neptune.ai/njoysubho/petal/
Remember to stop your project once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/project#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [29]:
model.to(device)

PetalModel(
  (model): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.

In [30]:
loss_func = nn.CrossEntropyLoss()

In [31]:
for iteration in range(params['iterations']):
    accuracy = 0
    correct = 0
    val_correct = 0
    val_accuracy =0
    for (x,y) in train_dl:
        x,y = x.to(device),y.to(device)
        prediction = model(x)
        optimizer.zero_grad()
        train_loss = loss_func(prediction,y)
        _, predicted = torch.max(prediction.data, 1)
        print("Prediceted {}. actual {}".format(predicted,y))
        correct += (predicted == y).float().sum()
        train_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        xm.optimizer_step(optimizer, barrier=True)
    accuracy = 100 * correct / len(train_dataset)
    run['metrics/train/accuracy'].log(accuracy)
    with torch.no_grad():
        for xVal,yVal in val_dl:
            xVal,yVal= xVal.to(device),yVal.to(device)
            val_prediction = model(xVal)
            _, val_predicted = torch.max(val_prediction.data, 1)
            val_correct += (val_predicted == yVal).float().sum()
    val_accuracy = 100 * val_correct / len(val_dataset)
    run['metrics/val/accuracy'].log(val_accuracy)
    print("Epoch {} Train Accuracy = {} Validation Accuracy = {}".format(iteration,accuracy,val_accuracy))


KeyboardInterrupt: 

In [ ]:
run.stop()

In [ ]:
#https://www.kaggle.com/code/piantic/pytorch-tpu/notebook
#https://medium.com/the-owl/installing-pytorch-xla-in-google-colab-without-35d37d1d03c6